In [1]:
import requests
import csv
import os
import pandas as pd
import json
import re
from datetime import datetime
import traceback
today_string = datetime.now().strftime('%Y-%m-%d')

In [2]:
input_path = "C:\\Users\\khan32\\Documents\\factgrid_recon"

## Export Csv Personendaten

In [5]:
input_file = f"WIAG-Domherren-DB-Lebensdaten_2024-07-10.csv"
input_path_file = os.path.join(input_path, input_file)
persons_df = pd.read_csv(input_path_file, sep=';')
print(len(persons_df))
persons_df.head()

15736


,id,corpus,givenname,prefix,familyname,displayname,date_of_birth,date_of_death,biographical_dates,summary_offices,GND_ID,GSN,FactGrid_ID,Wikidata_ID,Wikipedia
0,WIAG-Pers-EPISCGatz-03848-001,epc,Ulrich,NaN,NaN,Ulrich,NaN,973,+ 973,"Bischof Augsburg 923-973, Domherr Domstift Aug...",118625284,059-01621-001,Q653546,Q714866,Ulrich_von_Augsburg
1,WIAG-Pers-EPISCGatz-03858-001,epc,Siegfried,NaN,NaN,Siegfried,NaN,1096,+ 1096,"Ernannter Bischof Augsburg 1077-1096, Domherr ...",13805746X,050-06892-001,Q653421,Q64763639,NaN
2,WIAG-Pers-EPISCGatz-04027-001,epc,Norbert,NaN,NaN,Norbert,NaN,NaN,~ 1079/1080,Bischof Chur 1079/1080-1088,NaN,710-00332-001,Q653192,Q35136089,Norbert_von_Hohenwart
3,WIAG-Pers-CANON-91597-001,dreg-can,Sizo,NaN,NaN,Sizo,NaN,NaN,~ um 1099,Domkustos Domstift Augsburg um 1099,NaN,710-00420-001,Q700910,NaN,NaN
4,WIAG-Pers-EPISCGatz-02518-001,epc,Udalschalk,NaN,NaN,Udalschalk,NaN,1202,+ 1202,"Bischof Augsburg 1184-1202, Dompropst Domstift...",138018383,059-01616-001,Q649639,Q68911884,NaN


In [28]:
url = 'https://database.factgrid.de/sparql'
query = """
SELECT ?person ?wiag WHERE {
  ?person wdt:P601 ?wiag.
  ?person wdt:P2 wd:Q7.
}
"""

r = requests.get(url, params={'query': query}, headers={"Accept": "application/json"})
data = r.json()
wiag_ids_df = pd.json_normalize(data['results']['bindings'])

len(wiag_ids_df)

13150

In [29]:
# extract out q id
def extract_qid(df, column):
    df[column] = df[column].map(lambda x: x.strip('https://database.factgrid.de/entity/'))
 
#factgrid_df['item.value'] = factgrid_df['item.value'].map(lambda x: x.strip('https://database.factgrid.de/entity/'))

# drop irrelevant columns
def drop_type_columns(df):
    df.drop(columns=[column for column in df.columns if column.endswith('type')], inplace=True)
    df.drop(columns=[column for column in df.columns if column.endswith('xml:lang')], inplace=True)

In [30]:
drop_type_columns(wiag_ids_df)
extract_qid(wiag_ids_df, 'person.value')
wiag_ids_df.columns = ['FactGrid_ID', 'old_wiag_id']
wiag_ids_df

,FactGrid_ID,old_wiag_id
0,Q226,WIAG-Pers-EPISCGatz-12636-001
1,Q323,WIAG-Pers-EPISCGatz-12604-001
2,Q443,WIAG-Pers-EPISCGatz-10281-001
3,Q602,WIAG-Pers-EPISCGatz-12634-001
4,Q1285,WIAG-Pers-EPISCGatz-12655-001
...,...,...
13145,Q947671,WIAG-Pers-CANON-89050-001
13146,Q947672,WIAG-Pers-CANON-89051-001
13147,Q947673,WIAG-Pers-CANON-89053-001
13148,Q947674,WIAG-Pers-CANON-89054-001


In [31]:
merged_df = wiag_ids_df.merge(persons_df[['FactGrid_ID', 'id']], on='FactGrid_ID')
merged_df

,FactGrid_ID,old_wiag_id,id
0,Q226,WIAG-Pers-EPISCGatz-12636-001,WIAG-Pers-EPISCGatz-12636-001
1,Q323,WIAG-Pers-EPISCGatz-12604-001,WIAG-Pers-EPISCGatz-12604-001
2,Q1285,WIAG-Pers-EPISCGatz-12655-001,WIAG-Pers-EPISCGatz-12655-001
3,Q183870,WIAG-Pers-EPISCGatz-10081-001,WIAG-Pers-EPISCGatz-10081-001
4,Q234733,WIAG-Pers-CANON-15720-001,WIAG-Pers-CANON-15720-001
...,...,...,...
9103,Q947671,WIAG-Pers-CANON-89050-001,WIAG-Pers-CANON-89050-001
9104,Q947672,WIAG-Pers-CANON-89051-001,WIAG-Pers-CANON-89051-001
9105,Q947673,WIAG-Pers-CANON-89053-001,WIAG-Pers-CANON-89053-001
9106,Q947674,WIAG-Pers-CANON-89054-001,WIAG-Pers-CANON-89054-001


In [32]:
diff_df = merged_df[merged_df['old_wiag_id'] != merged_df['id']]
diff_df

,FactGrid_ID,old_wiag_id,id


In [33]:
fg_qs_csv = diff_df.copy()
fg_qs_csv.columns = ['qid', '-P601', 'P601']
fg_qs_csv['-P601'] = '"' + fg_qs_csv['-P601'] + '"'
fg_qs_csv['P601'] = '"' + fg_qs_csv['P601'] + '"'
fg_qs_csv.set_index('qid')
fg_qs_csv

,qid,-P601,P601


In [34]:
today_string = datetime.now().strftime('%Y-%m-%d')

In [35]:
fg_qs_csv.to_csv(f'factgrid_wiag_id_update_{today_string}.csv', index=False)

## Do a cyclic check for all entries in factgrid

In [36]:
from tqdm import tqdm
import aiohttp
import asyncio
import time
import ssl
import copy

In [37]:
print(len(pd.unique(wiag_ids_df['FactGrid_ID'])))
print(len(pd.unique(wiag_ids_df['old_wiag_id'])))
print(len(wiag_ids_df))

13150
13150
13150


In [38]:
wiag_ids_df

,FactGrid_ID,old_wiag_id
0,Q226,WIAG-Pers-EPISCGatz-12636-001
1,Q323,WIAG-Pers-EPISCGatz-12604-001
2,Q443,WIAG-Pers-EPISCGatz-10281-001
3,Q602,WIAG-Pers-EPISCGatz-12634-001
4,Q1285,WIAG-Pers-EPISCGatz-12655-001
...,...,...
13145,Q947671,WIAG-Pers-CANON-89050-001
13146,Q947672,WIAG-Pers-CANON-89051-001
13147,Q947673,WIAG-Pers-CANON-89053-001
13148,Q947674,WIAG-Pers-CANON-89054-001


In [39]:
missed = []
async def get(old_wiag_id, fg_id, session):
    global missed
    wiag_id = old_wiag_id
    try:
        async with session.get(url=f'https://wiag-vocab.adw-goe.de/id/{old_wiag_id}?format=Json') as response:
            data = await response.json()
            wiag_id = data['persons'][0]['wiagId']
            wiag_qid = data['persons'][0]['identifier']['Factgrid'].split('/')[-1]
            if wiag_qid != fg_id:
                print("Wiag linking to different factgrid id", old_wiag_id, fg_id, wiag_qid)
    except KeyError as e:
        ending_str = ''
        if wiag_id != old_wiag_id:
            ending_str = '(' + wiag_id + ')'
        print(f"Factgrid id {fg_id} not present for {old_wiag_id}{ending_str}.")
    except aiohttp.client_exceptions.ContentTypeError as e:
        missed.append([old_wiag_id, fg_id])
    except Exception as e:
        print(old_wiag_id, fg_id)
        print(traceback.format_exc())
        

async def main(urls):
    async with aiohttp.ClientSession() as session:
        chunk_size = 500
        for i in range(0, len(urls), chunk_size):
            try:
                await asyncio.gather(*(get(*url, session) for url in urls[i:i+chunk_size]))
                print(f"{i + chunk_size}/{len(urls)} checked. Missed count: {len(missed)}")
                time.sleep(1)
            except ssl.SSLError as e:
                i = i - 1
                print(f"Retrying last batch")
    print(f"Finalized all. Couldn't get data for {len(missed)} entries")

#urls = list(map(lambda x: f'https://wiag-vocab.adw-goe.de/id/{x}?format=Json', wiag_ids_df['old_wiag_id'].to_list()))

In [40]:
counter = 0
missed = list(zip(list(wiag_ids_df['old_wiag_id']), list(wiag_ids_df['FactGrid_ID'])))
while missed:
    counter += 1
    print(f"Starting attempt #{counter}")
    missed_copy = copy.deepcopy(missed)
    missed = []
    await main(missed_copy)

Starting attempt #1
500/13150 checked. Missed count: 0
1000/13150 checked. Missed count: 17
1500/13150 checked. Missed count: 166
2000/13150 checked. Missed count: 275
2500/13150 checked. Missed count: 373
3000/13150 checked. Missed count: 411
3500/13150 checked. Missed count: 579
4000/13150 checked. Missed count: 760
4500/13150 checked. Missed count: 923
5000/13150 checked. Missed count: 1053
Factgrid id Q704412 not present for WIAG-Pers-EPISCGatz-21774-001.
5500/13150 checked. Missed count: 1241
Factgrid id Q655561 not present for WIAG-Pers-EPISCGatz-03201-001.
6000/13150 checked. Missed count: 1374
6500/13150 checked. Missed count: 1534
7000/13150 checked. Missed count: 1682
7500/13150 checked. Missed count: 1720
8000/13150 checked. Missed count: 1860
8500/13150 checked. Missed count: 1987
9000/13150 checked. Missed count: 2106
9500/13150 checked. Missed count: 2145
10000/13150 checked. Missed count: 2266
10500/13150 checked. Missed count: 2396
11000/13150 checked. Missed count: 251

In [41]:
# for _, (fg_id, old_wiag_id) in tqdm(wiag_ids_df.iterrows()):
#     r = requests.get(f'https://wiag-vocab.adw-goe.de/id/{old_wiag_id}?format=Json')
#     data = r.json()
#     try:
#         wiag_qid = data['persons'][0]['identifier']['Factgrid'].split('/')[-1]
#         if wiag_qid != fg_id:
#             print(fg_id, old_wiag_id, wiag_qid)
#     except Exception as e:
#         print(data)


In [42]:
wiag_ids_df[wiag_ids_df['old_wiag_id'] == 'WIAG-Pers-CANON-89270-001']

,FactGrid_ID,old_wiag_id


In [43]:
persons_df[persons_df['id'] == 'WIAG-Pers-CANON-89270-001']

,id,corpus,givenname,prefix,familyname,displayname,date_of_birth,date_of_death,biographical_dates,summary_offices,GND_ID,GSN,FactGrid_ID,Wikidata_ID,Wikipedia


In [45]:
if [1]:
    print("something")
else:
    print("nothing")

something
